In [1]:
from collections import deque
from dataclasses import dataclass, field
from itertools import combinations, permutations
import json
from typing import Optional
import re

from pyprojroot import here

In [2]:
testInput = [
    'Valve AA has flow rate=0; tunnels lead to valves DD, II, BB',
    'Valve BB has flow rate=13; tunnels lead to valves CC, AA',
    'Valve CC has flow rate=2; tunnels lead to valves DD, BB,',
    'Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE',
    'Valve EE has flow rate=3; tunnels lead to valves FF, DD',
    'Valve FF has flow rate=0; tunnels lead to valves EE, GG',
    'Valve GG has flow rate=0; tunnels lead to valves FF, HH',
    'Valve HH has flow rate=22; tunnel leads to valve GG',
    'Valve II has flow rate=0; tunnels lead to valves AA, JJ',
    'Valve JJ has flow rate=21; tunnel leads to valve II'
]

In [3]:
(13 + 2 + 20 + 3 + 22 + 21) * 26

2106

In [4]:
@dataclass
class Valve:
    valveId: str
    flowRate: int
    childrenIds: list[str]

In [5]:
def calcDistance(graph: dict[str, Valve], valve1: Valve, valve2: Valve) -> int:

    @dataclass
    class Node:
        valve: Valve
        distanceTraveled: int = 0
        valvesVisited: set[str] = field(default_factory=set)

    queue = deque([Node(valve1)])
    while queue:
        node = queue.popleft()

        # valve1 and valve 2 are the same
        if node.valve.valveId == valve2.valveId:
            return 0

        # cycled
        if node.valve.valveId in node.valvesVisited:
            continue

        # directly connected
        if valve2.valveId in node.valve.childrenIds:
            return node.distanceTraveled + 1

        # indirectly connected
        for child in node.valve.childrenIds:
            childNode = Node(
                valve = graph[child],
                distanceTraveled = node.distanceTraveled + 1,
                valvesVisited = node.valvesVisited.union([node.valve.valveId])
            )
            queue.append(childNode)

    return -1


def calcDistances(graph: dict[str, Valve]) -> dict[str, dict[str, int]]:
    distances = {}

    for valve1 in graph.values():
        for valve2 in graph.values():
            distances.setdefault(valve1.valveId, {})[valve2.valveId] = calcDistance(graph, valve1, valve2)

    return distances

In [6]:
@dataclass
class Node:
    destValveA: Valve
    destValveB: Valve
    destDistA: int
    destDistB: int
    remainingValveIds: set[str]
    flowRate: int = 0
    pressureReleased: int = 0
    currentMinute: int = 1


def releaseMaxPressure(
    graph: dict[str, Valve],
    distances: dict[str, dict[str, int]],
    minutesAllowed: int,
    startingValveId: str
    ) -> int:
        
    maxPressureReleased = 0
    remainingValveIds = set([valve.valveId for valve in graph.values()])
    valveIdCombos = combinations(remainingValveIds, 2)
    stack: list[Node] = []

    for valveIdA, valveIdB in valveIdCombos:
        node = Node(
            destValveA = graph[valveIdA],
            destValveB = graph[valveIdB],
            destDistA = distances[startingValveId][valveIdA],
            destDistB = distances[startingValveId][valveIdB],
            remainingValveIds = remainingValveIds
        )
        stack.append(node)

    while stack:
        node = stack.pop()
        remainingValveIds = node.remainingValveIds - set([node.destValveA.valveId, node.destValveB.valveId])

        # time is up
        if node.currentMinute == minutesAllowed + 1:
            maxPressureReleased = max(maxPressureReleased, node.pressureReleased)
            continue

        # release pressure
        node.pressureReleased += node.flowRate
        node.currentMinute += 1

        # continue traveling A and continue traveling B
        # continue traveling A and open valve B
        # continue traveling B and open valve A
        # continue traveling A and do nothing B
        # continue traveling B and do nothing A
        # open valve A and do nothing B
        # open valve B and do nothing A

    
            
    return maxPressureReleased

In [7]:
path = here('./16/input.txt')
with open(path, 'r') as fp:
    lines = fp.readlines()
    # lines = testInput

data = [(*re.findall('([A-Z]{2}|\d+)(?!,|$)', line), re.findall('([A-Z]{2})(?=,|$)', line)) for line in lines]
valvesGraph = {valveId: Valve(valveId, int(flowRate), childrenIds) for valveId, flowRate, childrenIds in data}
distances = calcDistances(valvesGraph)
startingValveId = 'AA'
targetGraph = {valveId: valve for valveId, valve in valvesGraph.items() if valve.flowRate > 0}

# maxPressureReleased = releaseMaxPressure(targetGraph, distances, 26, startingValveId)
# print(maxPressureReleased)

In [8]:
[*targetGraph.keys()]

['EJ',
 'VO',
 'TE',
 'UP',
 'VR',
 'SE',
 'GL',
 'AR',
 'XN',
 'OW',
 'UM',
 'JF',
 'AH',
 'HZ',
 'TO']

In [13]:
def validPermutations(
    graph: dict[str, Valve],
    distances: dict[str, dict[str, int]],
    startingValveId = 'AA',
    ):

    @dataclass
    class Node:
        currentValveId: str
        remainingValveIds: set[str]
        timeRemaining: int


    timeRemaining = 30
    validPermutations = []
    stack = [Node(valveId, set(list(graph.keys())) - set([valveId]), 30) for valveId in graph.keys()]

    while stack:
        path = stack.pop()
        previousValveId = path[-2]
        currentValveId = path[-1]
        timeRemaining -= distances[previousValveId][currentValveId]
        remainingValveIds = [valveId for valveId in graph.keys() if valveId not in path]

        for nextValveId in remainingValveIds:
            if distances[currentValveId][nextValveId] < timeRemaining:
                stack.append(path + [nextValveId])
            else:
                validPermutations += 1

    return validPermutations

validPermutations(targetGraph, distances) 
    

809